# TABELAS SILVER

## users

In [0]:
%sql
  
CREATE OR REPLACE TABLE silver_users AS
SELECT
    -- data de criação (sem hora)
    TO_DATE(
        regexp_extract(creation_date, '^[A-Za-z]{3} [0-9]{1,2}, [0-9]{4}', 0),
        'MMM d, yyyy'
    ) AS creation_dt,

    -- data de modificação (sem hora)
    TO_DATE(
        regexp_extract(modified_date, '^[A-Za-z]{3} [0-9]{1,2}, [0-9]{4}', 0),
        'MMM d, yyyy'
    ) AS modified_dt,

    -- Identificador do usuário
    unique_id AS user_id,

    -- Campos básicos
    Nome AS nome,
    email,

    -- Telefone limpo
    regexp_replace(WhatsApp, '[^0-9]', '') AS telefone,

    -- CPF/CNPJ somente com dígitos
    regexp_replace(`CPF/CNPJ`, '[^0-9]', '') AS cpf_cnpj,

    -- Categoria de assinatura
    AssinaturaUsuario AS assinatura_usuario,

    -- Campos organizacionais
    equipeUser AS equipe,
    `escritórioUser` AS escritorio,
    relatoriosUser AS relatorios_user,

    -- Booleanos padronizados
    CASE 
        WHEN lower(AceiteContato) IN ('1','true','sim','yes','ativo') THEN true
        ELSE false
    END AS aceite_contato,

    CASE 
        WHEN lower(`Notificação`) IN ('1','true','sim','yes','ativo') THEN true
        ELSE false
    END AS notificacao,

    CASE 
        WHEN lower(StatusConta) IN ('ativo','1','true') THEN true
        ELSE false
    END AS status_conta

FROM workspace.default.bronze_users
WHERE _rescued_data IS NULL;


In [0]:
%sql
CREATE OR REPLACE TABLE silver_projetos AS
SELECT
    -- ID do projeto
    unique_id AS project_id,

    -- Campos textuais limpos
    nomeProjeto AS nome_projeto,
    `ambiente/servico` AS ambiente_servico,
    anexosProjeto AS anexos_projeto,
    desenhoProjeto AS desenho_projeto,

    -- Conversão de tipos numéricos
    TRY_CAST(
    regexp_replace(
        regexp_replace(Metragem, ',', '.'),
        '[^0-9\\.]', ''
    ) 
AS DOUBLE
) AS metragem,

    CAST(qntVisita AS INT)   AS qnt_visita,

-- data de criação (sem hora)
    TO_DATE(
        regexp_extract(creation_date, '^[A-Za-z]{3} [0-9]{1,2}, [0-9]{4}', 0),
        'MMM d, yyyy'
    ) AS creation_dt,

    -- data de modificação (sem hora)
    TO_DATE(
        regexp_extract(modified_date, '^[A-Za-z]{3} [0-9]{1,2}, [0-9]{4}', 0),
        'MMM d, yyyy'
    ) AS modified_dt,

    -- Atributos organizacionais
    arquivado,
    emailUserCriador AS email_user_criador,
    equipeProjeto    AS equipe_projeto,
    escritorio,
    userCriadorProjeto AS user_criador_projeto,
    visitasProjeto     AS visitas_projeto,
    zz_equipeUserProjeto AS equipe_user_projeto

FROM bronze_projetos
WHERE _rescued_data IS NULL;


In [0]:
%sql
CREATE OR REPLACE TABLE silver_visitas AS
SELECT
    -- Tags e classificações
    `ambiente/servicoTag` AS ambiente_servico_tag,
    TagAmbiente AS tag_ambiente,
    Tags AS tags,
    `TagServiço` AS tag_servico,

    -- data de criação (sem hora)
    TRY_TO_DATE(
        regexp_extract(creation_date, '^[A-Za-z]{3} [0-9]{1,2}, [0-9]{4}', 0),
        'MMM d, yyyy'
    ) AS creation_dt,

    -- data de modificação (sem hora)
    TRY_TO_DATE(
        regexp_extract(modified_date, '^[A-Za-z]{3} [0-9]{1,2}, [0-9]{4}', 0),
        'MMM d, yyyy'
    ) AS modified_dt,
    

    -- Atributos gerais
    Nome AS nome,
    escritorio,
    projetoVisita AS projeto_visita,
    statusVisita AS status_visita,
    userEquipe AS user_equipe,
    fotos,
    Creator AS creator,
    horaVisita AS hora_visita

FROM bronze_visitas
WHERE _rescued_data IS NULL;


In [0]:
%sql
SELECT project_id, metragem, qnt_visita
FROM silver_projetos
WHERE metragem IS NOT NULL
LIMIT 5;

In [0]:
%sql
SELECT creation_dt 
FROM workspace.default.silver_escritorios
LIMIT 10;

In [0]:
%sql
CREATE OR REPLACE TABLE workspace.default.silver_pendencias AS
SELECT
    CASE 
        WHEN lower(envolver_cliente) IN ('1','true','sim','yes','ativo') THEN true
        ELSE false
    END AS envolver_cliente,

    projetoPendencias AS projeto_pendencias,

    CASE 
        WHEN lower(Resolvida) IN ('1','true','sim','yes','ativo') THEN true
        ELSE false
    END AS resolvida,

    Titulo AS titulo,
    Creator AS creator,

    CASE 
        WHEN lower(Urgente) IN ('1','true','sim','yes','ativo') THEN true
        ELSE false
    END AS urgente,

    visitaPendencias AS visita_pendencias,


     -- data de criação (sem hora)
    TRY_TO_DATE(
        regexp_extract(creation_date, '^[A-Za-z]{3} [0-9]{1,2}, [0-9]{4}', 0),
        'MMM d, yyyy'
    ) AS creation_dt,

    -- data de modificação (sem hora)
    TRY_TO_DATE(
        regexp_extract(modified_date, '^[A-Za-z]{3} [0-9]{1,2}, [0-9]{4}', 0),
        'MMM d, yyyy'
    ) AS modified_dt  

FROM workspace.default.bronze_pendencias
WHERE _rescued_data IS NULL;



In [0]:
%sql
CREATE OR REPLACE TABLE silver_fotos AS
SELECT
    -- ID da foto
    id AS foto_id,

    -- Limpeza de campos textuais
    `ambiente/servicoTag` AS ambiente_servico_tag,
    imagem,
    imagem_URL AS imagem_url,
    marker,
    projeto,
    visita,

    -- Boolean padronizado
    CASE 
        WHEN lower(cliente) IN ('1','true','sim','yes','ativo') THEN true
        ELSE false
    END AS cliente,

    TRY_TO_DATE(
        regexp_extract(creation_date, '^[A-Za-z]{3} [0-9]{1,2}, [0-9]{4}', 0),
        'MMM d, yyyy'
    ) AS creation_dt,
    
    Creator AS creator

FROM workspace.default.bronze_fotos
WHERE _rescued_data IS NULL;


In [0]:
%sql
CREATE OR REPLACE TABLE silver_assinaturas AS
SELECT
    -- Informações do plano
    assinatura,
    ciclo,
    
    -- Limpeza de CPF/CNPJ: manter apenas números
    regexp_replace(`CPFouCNPJ`, '[^0-9]', '') AS cpf_ou_cnpj,

    -- Padronização de boolean
    CASE 
        WHEN lower(`CPFouCNPJ_valido`) IN ('1','true','sim','yes','valido','válido') THEN true
        ELSE false
    END AS cpf_ou_cnpj_valido,

    -- Atributos adicionais
    `escritorioAssinatura` AS escritorio_assinatura,
    `idAssinaturaStripe` AS id_assinatura_stripe,
    `textoAssinatura` AS texto_assinatura,

    -- Conversão de data corrigida:
    TRY_TO_DATE(
        regexp_extract(creation_date, '^[A-Za-z]{3} [0-9]{1,2}, [0-9]{4}', 0),
        'MMM d, yyyy'
    ) AS creation_dt,


    -- Metadados
        creator

FROM workspace.default.bronze_assinaturas
WHERE _rescued_data IS NULL;


In [0]:
%sql
CREATE OR REPLACE TABLE silver_escritorios AS
SELECT
    Empresa AS empresa,
    nomeEscritorio AS nome_escritorio,
    pagamentoEscritorio AS pagamento_escritorio,
    projetosEscritorio AS projetos_escritorio,
    `proprietárioUser` AS proprietario_user,
    usuariosEscritorio AS usuarios_escritorio,
    
    -- Conversão de data corrigida:
    TRY_TO_DATE(
        regexp_extract(creation_date, '^[A-Za-z]{3} [0-9]{1,2}, [0-9]{4}', 0),
        'MMM d, yyyy'
    ) AS creation_dt

FROM workspace.default.bronze_escritorios
WHERE _rescued_data IS NULL;


In [0]:
%sql
CREATE OR REPLACE TABLE silver_ambiente AS
SELECT
    -- Strings diretas
    Ambienteprojeto              AS ambiente_projeto,
    AmbienteUnicoProjeto         AS ambiente_unico_projeto,
    escritorioAmbiente           AS escritorio_ambiente,
    logo,
    Nome                         AS nome,
    Tag                          AS tag,
    zz_visitaAmbiente            AS visita_ambiente,
    Creator                      AS creator,

    -- Boolean padronizado
    CASE
        WHEN lower(AmbienteUniversal) IN ('1','true','sim','yes','y') THEN true
        ELSE false
    END AS ambiente_universal,

    -- Ordem já é INT na bronze
    Ordem AS ordem,

    -- Quantidade -> INT (limpeza total)
    CAST(
        regexp_replace(Quantidade, '[^0-9]', '')
        AS INT
    ) AS quantidade,

    TRY_TO_DATE(
        regexp_extract(creation_date, '^[A-Za-z]{3} [0-9]{1,2}, [0-9]{4}', 0),
        'MMM d, yyyy'
    ) AS creation_dt

FROM workspace.default.bronze_ambientes
WHERE _rescued_data IS NULL;


In [0]:
%sql
CREATE OR REPLACE TABLE silver_equipes AS
SELECT
    foto,
    nome_user,
    `permissãoEquipe` AS permissao_equipe,
    projetoEquipe     AS projeto_equipe,
    userEquipe        AS user_equipe,
    zz_email,

-- Conversão de data corrigida:
    TRY_TO_DATE(
        regexp_extract(creation_date, '^[A-Za-z]{3} [0-9]{1,2}, [0-9]{4}', 0),
        'MMM d, yyyy'
    ) AS creation_dt


FROM workspace.default.bronze_equipes
WHERE _rescued_data IS NULL;


In [0]:
%sql
select * from silver_ambiente_servicos
limit 10;

In [0]:
%sql
CREATE OR REPLACE TABLE silver_ambiente_servicos AS
SELECT
    logo,
    nome,
    nome2,
    observacao,
    projeto,

    -- Quantidade convertida para INT
    CAST(
        regexp_replace(quantidade, '[^0-9]', '')
        AS INT
    ) AS quantidade,

    tipo,

    -- Boolean padronizado
    CASE
        WHEN lower(urgente) IN ('1','true','sim','yes','y','urgente') THEN true
        ELSE false
    END AS urgente,

    visita,

    -- Conversão de data corrigida:
    TRY_TO_DATE(
        regexp_extract(creation_date, '^[A-Za-z]{3} [0-9]{1,2}, [0-9]{4}', 0),
        'MMM d, yyyy'
    ) AS creation_dt,

    -- data de modificação (sem hora)
    TRY_TO_DATE(
        regexp_extract(modified_date, '^[A-Za-z]{3} [0-9]{1,2}, [0-9]{4}', 0),
        'MMM d, yyyy'
    ) AS modified_dt,  

    Creator AS creator

FROM workspace.default.bronze_ambiente_servicos
WHERE _rescued_data IS NULL;
